<div style="padding:15px 20px 20px 20px;
            color:#004346;
            font-size:40px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#201E20;
            background-color: #E8F1F2;
            overflow:hidden;
            font-weight:400">
<p style="font-weight: bold; text-align: center;">SentenceTransformer</p>

</div>


[SentenceTransformers](https://www.sbert.net/) — это платформа Python для современного встраивания предложений, текста и изображений. 

<div style="padding:0px 20px 10px;
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#201E20;
            background-color: #E8F1F2;
            overflow:hidden;
            font-weight:400">

# Используемые библиотеки

</div>

In [1]:
from rich.console import Console
import os
import numpy as np
import pandas as pd
import random
import re
from sentence_transformers import SentenceTransformer, util

# константы
RANDOM_SEED = 42
BASE_PATH = 'data' # корневая директория

# настройки блокнота
pd.options.display.max_colwidth = 500
random.seed(RANDOM_SEED)

# экземпляр консоли
console = Console()

<div style="padding:0px 20px 10px;
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#201E20;
            background-color: #E8F1F2;
            overflow:hidden;
            font-weight:400">

# Анализ данных

</div>

## Загрузка данных и общая информация

In [2]:
# загрузка списока товаров, которые производит и распространяет заказчик
try:
    product = pd.read_csv(os.path.join(BASE_PATH, 'marketing_product.csv'), sep=';')
    console.print('[bold]Список товаров:[/bold]')
    display(product.head(2))
    display(product.info())
except:
    print('Информация о товарах, которые производит и распространяет заказчик, недоступна')
    
# загрузка результатов работы парсера площадок дилеров
try:
    dealer = pd.read_csv(os.path.join(BASE_PATH, 'marketing_dealerprice.csv'), sep=';')
    console.print('[bold]Парсинг площадок дилеров:[/bold]')
    display(dealer.head(2))
    display(dealer.info())
except:
    print('Информация о результатах работы парсера площадок дилеров, недоступна')
    
# загрузка таблицы матчинга товаров заказчика и товаров дилеров
try:
    marketing = pd.read_csv(os.path.join(BASE_PATH, 'marketing_productdealerkey.csv'), sep=';')
    console.print('[bold]Матчинга товаров:[/bold]')
    display(marketing.head(2))
    display(marketing.info())
except:
    print('Информация о матчинге товаров, недоступна')

Список товаров:

,Unnamed: 0,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article,wb_article_td
0,0,245,008-1,4.680008e+12,Антисептик невымываемыйPROSEPT ULTRAконцентрат 1:10 / 1 л,360.00,858.0,20.0,"Антисептик невымываемый для ответственных конструкций PROSEPT ULTRA, концентрат, 1 л.","Антисептик невымываемый для ответственных конструкций PROSEPT ULTRA, концентрат, 1 л.","Антисептик невымываемый для ответственных конструкций PROSEPT ULTRA, концентрат, 1 л.",189522705.0,150033482.0,008-1,NaN
1,1,3,242-12,NaN,Антигололед - 32 PROSEPTготовый состав / 12 кг,460.16,1075.0,NaN,NaN,Антигололед - 32 PROSEPTготовый состав / 12 кг,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         496 non-null    int64  
 1   id                 496 non-null    int64  
 2   article            496 non-null    object 
 3   ean_13             464 non-null    float64
 4   name               494 non-null    object 
 5   cost               491 non-null    float64
 6   recommended_price  491 non-null    float64
 7   category_id        447 non-null    float64
 8   ozon_name          458 non-null    object 
 9   name_1c            485 non-null    object 
 10  wb_name            455 non-null    object 
 11  ozon_article       365 non-null    float64
 12  wb_article         340 non-null    float64
 13  ym_article         337 non-null    object 
 14  wb_article_td      32 non-null     object 
dtypes: float64(6), int64(2), object(7)
memory usage: 58.2+ KB


None

Парсинг площадок дилеров:

...

None

Матчинга товаров:

,id,key,dealer_id,product_id
0,1,546227,2,12
1,2,651265,2,106


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1700 entries, 0 to 1699
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1700 non-null   int64 
 1   key         1700 non-null   object
 2   dealer_id   1700 non-null   int64 
 3   product_id  1700 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 53.2+ KB


None

## product

In [3]:
df_product = product[['id', 'name']].dropna().drop_duplicates()

In [5]:
display(df_product.head())

,id,name
0,245,Антисептик невымываемыйPROSEPT ULTRAконцентрат 1:10 / 1 л
1,3,Антигололед - 32 PROSEPTготовый состав / 12 кг
2,443,"Герметик акриловый цвет сосна, ф/п 600мл"
3,147,Кондиционер для белья с ароматом королевского ИрисаCrystal Rinserконцентрат / 2 л
4,502,"Герметик акриловой цвет Белый, 7 кг"


## dealer

In [6]:
df_dealer = dealer[['product_key', 'product_name']].copy()
display(df_dealer.head())

,product_key,product_name
0,546227,"Средство универсальное Prosept Universal Spray, 500мл"
1,546408,"Концентрат Prosept Multipower для мытья полов, цитрус 1л"
2,546234,"Средство для чистки люстр Prosept Universal Anti-dust, 500мл"
3,651258,"Удалитель ржавчины PROSEPT RUST REMOVER 0,5л 023-05"
4,546355,Средство моющее для бани и сауны Prosept Multipower Wood 1л


## Формирование датасета для тестирования

In [7]:
df = pd.merge(df_dealer, marketing, left_on='product_key', right_on='key', how='inner')
df = pd.merge(df, df_product, left_on='product_id', right_on='id', how='left')
df = df.drop(['id_x', 'id_y', 'key'], axis=1).drop_duplicates()
df.head()

,product_key,product_name,dealer_id,product_id,name
0,546227,"Средство универсальное Prosept Universal Spray, 500мл",2,12,"Универсальное моющее и чистящее средствоUniversal Sprayготовый состав / 0,5 л"
6,546408,"Концентрат Prosept Multipower для мытья полов, цитрус 1л",2,38,Концентрат для мытья половMULTIPOWER с ароматом цитрусаконцентрат / 1 л
18,546234,"Средство для чистки люстр Prosept Universal Anti-dust, 500мл",2,18,"Средство для чистки люстр Universal Anti-dust готовый состав / 0,5 л"
30,651258,"Удалитель ржавчины PROSEPT RUST REMOVER 0,5л 023-05",2,403,"Удалитель ржавчины RUST REMOVER готовый состав / 0,5 л"
42,546355,Средство моющее для бани и сауны Prosept Multipower Wood 1л,2,39,Моющее средство для бани и сауныMultipower Wood концентрат / 1 л


<div style="padding:0px 20px 10px;
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#201E20;
            background-color: #E8F1F2;
            overflow:hidden;
            font-weight:400">

# Очистка данных

</div>

In [8]:
SPACES = r'(?<=[а-яА-Я])(?=[a-zA-Z])|(?<=[a-zA-Z])(?=[а-яА-Я])'
DUP_SPACES = r'([ ])\1+'
COMMA_OUT_LINE = r',\s'
DROP_BRASKET = r'[()\s]'
DROP_NUM = r'(?<=\s)\d+(?=\s)'

In [9]:
def clean_string(input_string):
    input_string = re.sub(SPACES, ' ', input_string.lower()) # нижний регистр, обработка пропущенных пробелов
    input_string = re.sub(COMMA_OUT_LINE, ' ', input_string) # убираем внешние запятые
    input_string = re.sub(DROP_BRASKET, ' ', input_string) # убираем '(',')'
    input_string = input_string.replace('prosept', '') # убираем название фирмы
    input_string = input_string.replace('просепт', '')
    input_string = input_string.replace('"', ' ')
    input_string = input_string.replace('–', ' ') ## 0
    input_string = input_string.replace(' /', '') 
    input_string = input_string.replace('-', ' ')
    input_string = re.sub(DUP_SPACES, r'\1', input_string) # обработка двойных пробелов
    return input_string

In [10]:
df_product['name_clean'] = product['name'].astype(str).apply(clean_string)
df['product_name_clean'] = df['product_name'].astype(str).apply(clean_string)
df['name_clean'] = df['name'].astype(str).apply(clean_string)

In [11]:
df_product.head()

,id,name,name_clean
0,245,Антисептик невымываемыйPROSEPT ULTRAконцентрат 1:10 / 1 л,антисептик невымываемый ultra концентрат 1:10 1 л
1,3,Антигололед - 32 PROSEPTготовый состав / 12 кг,антигололед 32 готовый состав 12 кг
2,443,"Герметик акриловый цвет сосна, ф/п 600мл",герметик акриловый цвет сосна ф/п 600мл
3,147,Кондиционер для белья с ароматом королевского ИрисаCrystal Rinserконцентрат / 2 л,кондиционер для белья с ароматом королевского ириса crystal rinser концентрат 2 л
4,502,"Герметик акриловой цвет Белый, 7 кг",герметик акриловой цвет белый 7 кг


In [12]:
df.head()

,product_key,product_name,dealer_id,product_id,name,product_name_clean,name_clean
0,546227,"Средство универсальное Prosept Universal Spray, 500мл",2,12,"Универсальное моющее и чистящее средствоUniversal Sprayготовый состав / 0,5 л",средство универсальное universal spray 500мл,"универсальное моющее и чистящее средство universal spray готовый состав 0,5 л"
6,546408,"Концентрат Prosept Multipower для мытья полов, цитрус 1л",2,38,Концентрат для мытья половMULTIPOWER с ароматом цитрусаконцентрат / 1 л,концентрат multipower для мытья полов цитрус 1л,концентрат для мытья полов multipower с ароматом цитрусаконцентрат 1 л
18,546234,"Средство для чистки люстр Prosept Universal Anti-dust, 500мл",2,18,"Средство для чистки люстр Universal Anti-dust готовый состав / 0,5 л",средство для чистки люстр universal anti dust 500мл,"средство для чистки люстр universal anti dust готовый состав 0,5 л"
30,651258,"Удалитель ржавчины PROSEPT RUST REMOVER 0,5л 023-05",2,403,"Удалитель ржавчины RUST REMOVER готовый состав / 0,5 л","удалитель ржавчины rust remover 0,5л 023 05","удалитель ржавчины rust remover готовый состав 0,5 л"
42,546355,Средство моющее для бани и сауны Prosept Multipower Wood 1л,2,39,Моющее средство для бани и сауныMultipower Wood концентрат / 1 л,средство моющее для бани и сауны multipower wood 1л,моющее средство для бани и сауны multipower wood концентрат 1 л


<div style="padding:0px 20px 10px;
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#201E20;
            background-color: #E8F1F2;
            overflow:hidden;
            font-weight:400">

# ML

</div>

In [13]:
# предобученная модель
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [14]:
# получение векторных представлений для названий товаров
name_embeddings = model.encode(df_product['name_clean'].values)

In [15]:
df_model = df_product.copy()

In [16]:
# добавление нового столбца с векторными представлениями в DataFrame
df_model['name_embedding'] = list(name_embeddings)

In [17]:
df_model = df_model[['name', 'name_clean', 'name_embedding']]

__Тестирование__

In [18]:
def conclusion(input_sentence, n):
    input_embedding = model.encode(input_sentence)
    df = df_model.copy()
    df['input_vector'] = [input_embedding]*len(df)
    df['cos_sim'] = df.apply(lambda x: util.cos_sim(x['name_embedding'], x['input_vector']).item(), axis=1)
    df = df.sort_values(by='cos_sim', ascending=False)
    return df.head(n)['name'].values

In [19]:
def rez(x, n):
    predict = conclusion(x['product_name'], n)
    return 1 if x['name'] in predict else 0

In [20]:
# метрика для разных n
for n in [1, 3, 5, 10]:
    df['rez'] = df.apply(lambda x: rez(x, n), axis=1)
    print(f"accuracy@{n}: {df['rez'].mean()}")

accuracy@1: 0.6972205795387345
accuracy@3: 0.8521584861028977
accuracy@5: 0.8965109402720284
accuracy@10: 0.9343583678296866


__Посмотрим на записи, к которым не удалось подобрать товар из базы:__

In [26]:
display(df[df['rez']==0][['name', 'product_name']].head())

,name,product_name
90,"Шпатлевка-замазка для заделки глубоких выбоин и трещинPlastix / 1,4 кг","Шпаклевка выравнивающая акриловая PROSEPT Plastix белая, 1 кг."
212,"Шпатлевка для плит OSBProplast / 1,3 кг","Шпаклевка для OSB-плит акриловая PROSEPT Proplast белая, 1 кг."
800,"Антисептик лессирующий BiO LASUR / тик / 2,7 л",Просепт Bio Lasur антисептик лессирующий защитно-декоративный (9 л)
917,"Антисептик лессирующий BiO LASUR / белый люкс / 0,9 л",Просепт Bio Lasur антисептик лессирующий защитно-декоративный (900 мл)
930,"Антисептик лессирующий BiO LASUR / сосна / 0,9 л",Просепт Bio Lasur антисептик лессирующий защитно-декоративный (900 мл)
